<a href="https://colab.research.google.com/github/sidrusiya/AILA_Tasks/blob/main/Task2_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os
import nltk
import re
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dataset = open('/content/drive/MyDrive/Task2/task2_headnotes.json', 'r')
data = json.load(dataset)

In [ ]:
data

In [ ]:
yz = pd.DataFrame.from_dict(data, orient='index')

In [ ]:
dataset1 = open('/content/drive/MyDrive/Task2/task2_judgements.json', 'r')
data1 = json.load(dataset1)

In [ ]:
yz1 = pd.DataFrame.from_dict(data1, orient='index')
yz1

In [ ]:
df = pd.DataFrame(columns=['judge_id', 'id', 'sentence', 'relevance'])
for i in range(2545):
  for j in range(500):
    if yz1[i][j] is not None:
      df = df.append({'judge_id': j+1, 'id': yz1[i][j]['id'], 'sentence': yz1[i][j]['sentence'], 'relevance': yz1[i][j]['relevance']}, ignore_index=True)


In [ ]:
df

In [ ]:
import seaborn as sns
import gensim
nltk.download('all')
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, Dense, Dropout, Bidirectional, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from google.colab import drive
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
!pip3 install tensorflow_text
import tensorflow_text
!pip install -q tf-models-official
!pip install -q -U tensorflow-text
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab
import tensorflow_hub as hub
import tensorflow_text as tf_tex

In [ ]:
# Remove Excess [Size:: ] from EoS
def remove_size(text):
	result = re.search("\[size", text)
	if result is not None:
		text = text[:result.start()]
	return text.strip()

months = ['january', 'jan', 'feb', 'mar', 'apr', 'aug', 'sept', 'oct', 'nov', 'dec', 'feburary', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december', 'on']
def remove_months(text):
    text = text.split()
    text = [word for word in text if word.lower() not in set(months)]
    text = ' '.join(text)
    return text

def remove_nums(text):
    result = re.sub(r'[0-9+]', '', text)
    return result

def remove_comma(text):
    result = re.sub(r',', '', text)
    return result

In [ ]:
import sys
text = []
lem_text = []
for i in df.index:
    df['sentence'][i] = remove_size(df['sentence'][i])
    df['sentence'][i] = remove_months(df['sentence'][i])
    df['sentence'][i] = remove_nums(df['sentence'][i])
    df['sentence'][i] = remove_comma(df['sentence'][i])
    temp = df['sentence'][i].lower().split()
    clean = [word for word in temp if word not in stopwords.words('english')]
    clean = " ".join(clean)
    text.append(clean)
    lem_text.append(clean)

In [ ]:
for i in range(len(text)):
    text[i] = text[i].split()

for i in range(len(lem_text)):
    lem_text[i] = lem_text[i].split()

from nltk import PorterStemmer
st = PorterStemmer()
for i in range(len(text)):
    text[i] = [st.stem(word) for word in text[i]]

from nltk.stem import WordNetLemmatizer
  
lemmatizer = WordNetLemmatizer()

for i in range(len(lem_text)):
    lem_text[i] = [lemmatizer.lemmatize(word) for word in lem_text[i]]

for i in range(len(text)):
    text[i] = " ".join(text[i])

text[0]

for i in range(len(lem_text)):
    lem_text[i] = " ".join(lem_text[i])

lem_text[0]

df['lem_text'] = np.array(lem_text)
df['stem_text'] = np.array(text)

df

In [ ]:
df.relevance.value_counts()

In [ ]:
df.to_csv('nit_agartala_nlp_team_9.tsv', sep='\t', index=False)